In [1]:
# load in libraries
import pandas as pd
import os

In [2]:
# read in airports data
airports_dir = os.path.join(os.getcwd(), '../data/raw_data/our_airports_raw/airports.csv')
airports = pd.read_csv(
    airports_dir,
    usecols=[  # these columns exist in the dataset
        'id',
        'ident',
        'type',
        'name',
        'latitude_deg',
        'longitude_deg',
        'elevation_ft',
        'iso_country',
        'iso_region',
        'municipality',
        'iata_code'
    ],
    encoding='utf-8'
)
airports = airports.rename(columns={'id': 'airport_id'})

In [3]:
# view data
print(airports.head())

   airport_id ident           type                  name  latitude_deg  \
0        6523   00A       heliport     Total RF Heliport     40.070985   
1      323361  00AA  small_airport  Aero B Ranch Airport     38.704022   
2        6524  00AK  small_airport          Lowell Field     59.947733   
3        6525  00AL  small_airport          Epps Airpark     34.864799   
4      506791  00AN  small_airport  Katmai Lodge Airport     59.093287   

   longitude_deg  elevation_ft iso_country iso_region  municipality iata_code  
0     -74.933689          11.0          US      US-PA      Bensalem       NaN  
1    -101.473911        3435.0          US      US-KS         Leoti       NaN  
2    -151.692524         450.0          US      US-AK  Anchor Point       NaN  
3     -86.770302         820.0          US      US-AL       Harvest       NaN  
4    -156.456699          80.0          US      US-AK   King Salmon       NaN  


In [4]:
# read in runways data
runways_dir = os.path.join(os.getcwd(), '../data/raw_data/our_airports_raw/runways.csv')
runways = pd.read_csv(
    runways_dir,
    usecols=[  # these columns exist in the dataset
        'id', 
        'airport_ident', 
        'length_ft', 
        'width_ft', 
        'surface'
    ],
    encoding='utf-8'
)
runways = runways.rename(columns={'id': 'runways_id'})

In [5]:
# view data
print(runways.head())

   runways_id airport_ident  length_ft  width_ft surface
0      269408           00A       80.0      80.0  ASPH-G
1      255155          00AK     2500.0      70.0    GRVL
2      254165          00AL     2300.0     200.0    TURF
3      506792          00AN     4517.0      60.0     GVL
4      322128          00AS     1450.0      60.0    Turf


In [6]:
# Read in countries data
countries_dir = os.path.join(os.getcwd(), '../data/raw_data/our_airports_raw/countries.csv')
countries = pd.read_csv(
    countries_dir,
    usecols=[  # these columns exist in the dataset
        'id', 
        'code', 
        'name'
    ],
    encoding='utf-8'
)
countries = countries.rename(columns={'id': 'countries_id'})

In [7]:
# view
print(countries.head())

   countries_id code                  name
0        302672   AD               Andorra
1        302618   AE  United Arab Emirates
2        302619   AF           Afghanistan
3        302722   AG   Antigua and Barbuda
4        302723   AI              Anguilla


In [8]:
# Construct the file path
regions_dir = os.path.join(os.getcwd(), '../data/raw_data/our_airports_raw/regions.csv')

# Read the CSV with selected columns
regions = pd.read_csv(
    regions_dir,
    usecols=[
        'id',
        'code',
        'name',
        'iso_country'
    ],
    encoding='utf-8'
)
regions = regions.rename(columns={'id': 'regions_id'})

In [9]:
# Display first few rows to verify
print(regions.head())

   regions_id   code                        name iso_country
0      302811  AD-02              Canillo Parish          AD
1      302812  AD-03               Encamp Parish          AD
2      302813  AD-04           La Massana Parish          AD
3      302814  AD-05               Ordino Parish          AD
4      302815  AD-06  Sant Julià de Lòria Parish          AD


In [10]:
# Step 1: Join airports with runways on ident = airport_ident
airports_joined = pd.merge(
    airports,
    runways,
    left_on='ident',
    right_on='airport_ident',
    how='left'
)

In [11]:
# Step 2: Join with countries on iso_country = code with suffixes
airports_joined = pd.merge(
    airports_joined,
    countries[['code', 'name']],
    left_on='iso_country',
    right_on='code',
    how='left',
    suffixes=('', '_country')
)

# Rename 'name_country' to 'country_name'
airports_joined = airports_joined.rename(columns={'name_country': 'country_name'})


In [12]:
# Step 3: Join with regions on iso_region = code with suffixes
airports_joined = pd.merge(
    airports_joined,
    regions[['code', 'name']],
    left_on='iso_region',
    right_on='code',
    how='left',
    suffixes=('', '_region')
)

# Rename 'name_region' to 'region_name'
airports_joined = airports_joined.rename(columns={'name_region': 'region_name'})

In [13]:
# Step 4: Select required columns
airports_joined = airports_joined[[
    'ident',
    'iata_code',
    'type',
    'name',          # airport name (no suffix)
    'latitude_deg',
    'longitude_deg',
    'elevation_ft',
    'length_ft',
    'width_ft',
    'surface',
    'country_name',
    'region_name'
]]

In [15]:
# view
print(airports_joined.head())

  ident iata_code           type                  name  latitude_deg  \
0   00A       NaN       heliport     Total RF Heliport     40.070985   
1  00AA       NaN  small_airport  Aero B Ranch Airport     38.704022   
2  00AK       NaN  small_airport          Lowell Field     59.947733   
3  00AL       NaN  small_airport          Epps Airpark     34.864799   
4  00AN       NaN  small_airport  Katmai Lodge Airport     59.093287   

   longitude_deg  elevation_ft  length_ft  width_ft surface   country_name  \
0     -74.933689          11.0       80.0      80.0  ASPH-G  United States   
1    -101.473911        3435.0        NaN       NaN     NaN  United States   
2    -151.692524         450.0     2500.0      70.0    GRVL  United States   
3     -86.770302         820.0     2300.0     200.0    TURF  United States   
4    -156.456699          80.0     4517.0      60.0     GVL  United States   

    region_name  
0  Pennsylvania  
1        Kansas  
2        Alaska  
3       Alabama  
4       